# Análisis de Sentimiento para Predicción Financiera

## Objetivo
Analizar el sentimiento de noticias, redes sociales y foros sobre productos financieros (Oro, Dólar/Soles, Cobre) y correlacionarlo con indicadores económicos para generar predicciones.

## Factores Analizados
- Sentimiento de noticias financieras
- Tendencias en redes sociales
- Tipo de cambio USD/PEN (Dólar/Soles)
- Riesgo país
- Índice de confianza del consumidor

## Referencia
Inspirado en: Political Persuasion Analysis - Business Analytics

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Para análisis de texto
try:
    from textblob import TextBlob
    TEXTBLOB_AVAILABLE = True
except:
    TEXTBLOB_AVAILABLE = False
    print("⚠️ TextBlob no disponible. Usando análisis simulado.")
    print("   Para instalar: pip install textblob")

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Librerías importadas")

## 2. Descarga de Datos Financieros Reales

In [ ]:
# Período de análisis (últimos 2 años)
fecha_fin = datetime.now()
fecha_inicio = fecha_fin - timedelta(days=730)

print("=" * 70)
print("📊 DESCARGANDO DATOS FINANCIEROS")
print("=" * 70)
print(f"Período: {fecha_inicio.date()} a {fecha_fin.date()}")

# Productos financieros a analizar
SIMBOLOS = {
    'GC=F': 'Oro',
    'HG=F': 'Cobre',
    'PEN=X': 'USD_PEN',  # Dólar vs Sol Peruano
    'DX-Y.NYB': 'Dolar_Index',
    '^GSPC': 'SP500',
    'BTC-USD': 'Bitcoin'
}

# Descargar datos
datos = {}
for simbolo, nombre in SIMBOLOS.items():
    try:
        df = yf.download(simbolo, start=fecha_inicio, end=fecha_fin, progress=False)
        if len(df) > 0:
            datos[nombre] = df['Close']
            print(f"✅ {nombre}: {len(df)} registros")
    except Exception as e:
        print(f"❌ {nombre}: Error")

# Crear DataFrame
df_financiero = pd.DataFrame(datos)
df_financiero = df_financiero.dropna()

print(f"\nTotal de registros: {len(df_financiero)}")

## 3. Generación de Datos de Sentimiento

Simulamos datos de sentimiento basados en noticias, redes sociales y foros.
En producción, esto vendría de APIs de Twitter, NewsAPI, Reddit, etc.

In [ ]:
# Generar datos de sentimiento correlacionados con precios
np.random.seed(42)

print("=" * 70)
print("📰 GENERANDO DATOS DE SENTIMIENTO")
print("=" * 70)

# Noticias de ejemplo sobre el oro
NOTICIAS_ORO = [
    "Gold prices surge amid global uncertainty and inflation fears",
    "Investors flock to gold as safe haven asset",
    "Gold drops as dollar strengthens on Fed rate hike",
    "Central banks increase gold reserves significantly",
    "Gold faces pressure from rising Treasury yields",
    "Geopolitical tensions boost gold demand",
    "Gold miners report strong quarterly earnings",
    "Gold prices hit record high on economic worries",
    "Strong dollar weighs on gold prices",
    "Gold outlook remains bullish for 2024",
    "Physical gold demand surges in Asia",
    "Gold ETF inflows reach multi-year high",
    "Analysts raise gold price targets",
    "Gold consolidates near key support level",
    "Inflation data supports gold rally"
]

# Función para calcular sentimiento
def calcular_sentimiento(texto):
    """Calcular sentimiento de un texto (-1 a 1)"""
    if TEXTBLOB_AVAILABLE:
        return TextBlob(texto).sentiment.polarity
    else:
        # Análisis simple basado en palabras clave
        palabras_positivas = ['surge', 'boost', 'high', 'strong', 'bullish', 'raise', 'increase', 'rally', 'support']
        palabras_negativas = ['drop', 'pressure', 'weighs', 'fears', 'worry', 'decline', 'fall']
        
        texto_lower = texto.lower()
        pos = sum(1 for p in palabras_positivas if p in texto_lower)
        neg = sum(1 for p in palabras_negativas if p in texto_lower)
        
        if pos + neg == 0:
            return 0
        return (pos - neg) / (pos + neg)

# Calcular sentimiento base de noticias
sentimientos_base = [calcular_sentimiento(n) for n in NOTICIAS_ORO]
print(f"\nSentimiento promedio de noticias: {np.mean(sentimientos_base):.3f}")

In [ ]:
# Generar series temporales de sentimiento correlacionadas con precios
n_dias = len(df_financiero)

# Sentimiento correlacionado con retornos del oro
if 'Oro' in df_financiero.columns:
    retornos_oro = df_financiero['Oro'].pct_change().fillna(0)
    
    # Sentimiento de noticias (correlacionado con retornos)
    df_financiero['Sentimiento_Noticias'] = (
        retornos_oro * 10 + np.random.normal(0, 0.1, n_dias)
    ).clip(-1, 1)
    
    # Sentimiento de redes sociales (más volátil)
    df_financiero['Sentimiento_RRSS'] = (
        retornos_oro * 15 + np.random.normal(0, 0.2, n_dias)
    ).clip(-1, 1)
    
    # Sentimiento de foros (con lag)
    df_financiero['Sentimiento_Foros'] = (
        retornos_oro.shift(1).fillna(0) * 8 + np.random.normal(0, 0.15, n_dias)
    ).clip(-1, 1)

# Indicadores económicos simulados pero realistas
# Riesgo país (EMBI spread - típico de países latinoamericanos)
df_financiero['Riesgo_Pais'] = 150 + np.cumsum(np.random.normal(0, 5, n_dias))
df_financiero['Riesgo_Pais'] = df_financiero['Riesgo_Pais'].clip(80, 400)

# Índice de confianza del consumidor (0-100)
df_financiero['Indice_Confianza'] = 65 + np.cumsum(np.random.normal(0, 0.5, n_dias))
df_financiero['Indice_Confianza'] = df_financiero['Indice_Confianza'].clip(30, 100)

# Volumen de búsquedas en Google (índice 0-100)
df_financiero['Google_Trends_Oro'] = 50 + np.random.normal(0, 15, n_dias)
if 'Oro' in df_financiero.columns:
    df_financiero['Google_Trends_Oro'] += retornos_oro * 500
df_financiero['Google_Trends_Oro'] = df_financiero['Google_Trends_Oro'].clip(0, 100)

print("\n✅ Datos de sentimiento generados")
print(f"\nColumnas disponibles:")
for col in df_financiero.columns:
    print(f"  - {col}")

In [ ]:
# Estadísticas de sentimiento
print("\n" + "=" * 70)
print("ESTADÍSTICAS DE SENTIMIENTO")
print("=" * 70)

columnas_sentimiento = ['Sentimiento_Noticias', 'Sentimiento_RRSS', 'Sentimiento_Foros']
for col in columnas_sentimiento:
    if col in df_financiero.columns:
        print(f"\n{col}:")
        print(f"  Media: {df_financiero[col].mean():.3f}")
        print(f"  Std: {df_financiero[col].std():.3f}")
        print(f"  Min: {df_financiero[col].min():.3f}")
        print(f"  Max: {df_financiero[col].max():.3f}")

## 4. Visualización de Datos

In [ ]:
# Visualización de precios y sentimiento
fig, axes = plt.subplots(4, 1, figsize=(15, 14), sharex=True)

# 1. Precio del Oro
if 'Oro' in df_financiero.columns:
    axes[0].plot(df_financiero.index, df_financiero['Oro'], color='gold', linewidth=2)
    axes[0].set_ylabel('Precio (USD)')
    axes[0].set_title('Precio del Oro', fontweight='bold')
    axes[0].grid(True, alpha=0.3)

# 2. Sentimientos
for col, color in [('Sentimiento_Noticias', 'blue'), ('Sentimiento_RRSS', 'green'), ('Sentimiento_Foros', 'purple')]:
    if col in df_financiero.columns:
        axes[1].plot(df_financiero.index, df_financiero[col], label=col.replace('_', ' '), alpha=0.7, linewidth=1)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1].set_ylabel('Sentimiento (-1 a 1)')
axes[1].set_title('Análisis de Sentimiento', fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# 3. Riesgo País
if 'Riesgo_Pais' in df_financiero.columns:
    axes[2].plot(df_financiero.index, df_financiero['Riesgo_Pais'], color='red', linewidth=2)
    axes[2].set_ylabel('Puntos Básicos')
    axes[2].set_title('Riesgo País (EMBI Spread)', fontweight='bold')
    axes[2].grid(True, alpha=0.3)

# 4. Índice de Confianza
if 'Indice_Confianza' in df_financiero.columns:
    axes[3].plot(df_financiero.index, df_financiero['Indice_Confianza'], color='teal', linewidth=2)
    axes[3].set_ylabel('Índice (0-100)')
    axes[3].set_title('Índice de Confianza del Consumidor', fontweight='bold')
    axes[3].set_xlabel('Fecha')
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Tipo de cambio USD/PEN si está disponible
if 'USD_PEN' in df_financiero.columns:
    fig, axes = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
    
    axes[0].plot(df_financiero.index, df_financiero['USD_PEN'], color='green', linewidth=2)
    axes[0].set_ylabel('USD/PEN')
    axes[0].set_title('Tipo de Cambio Dólar/Sol Peruano', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Correlación con Oro
    if 'Oro' in df_financiero.columns:
        ax2 = axes[1]
        ax2.scatter(df_financiero['USD_PEN'], df_financiero['Oro'], alpha=0.5, s=20)
        ax2.set_xlabel('USD/PEN')
        ax2.set_ylabel('Precio del Oro (USD)')
        ax2.set_title('Correlación: Tipo de Cambio vs Oro', fontweight='bold')
        
        corr = df_financiero['USD_PEN'].corr(df_financiero['Oro'])
        ax2.text(0.05, 0.95, f'Correlación: {corr:.3f}', transform=ax2.transAxes, 
                fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. Análisis de Correlaciones

In [ ]:
# Matriz de correlación
print("=" * 70)
print("ANÁLISIS DE CORRELACIONES")
print("=" * 70)

# Seleccionar columnas para correlación
columnas_analisis = []
for col in ['Oro', 'Cobre', 'USD_PEN', 'Dolar_Index', 'SP500', 'Bitcoin',
            'Sentimiento_Noticias', 'Sentimiento_RRSS', 'Sentimiento_Foros',
            'Riesgo_Pais', 'Indice_Confianza', 'Google_Trends_Oro']:
    if col in df_financiero.columns:
        columnas_analisis.append(col)

corr_matrix = df_financiero[columnas_analisis].corr()

# Visualizar
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0, fmt='.2f',
            square=True, linewidths=0.5)
plt.title('Matriz de Correlación: Sentimiento y Factores Financieros', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Correlaciones clave con el Oro
if 'Oro' in corr_matrix.columns:
    print("\n📊 Correlaciones con el precio del Oro:")
    correlaciones_oro = corr_matrix['Oro'].sort_values(ascending=False)
    
    for factor, corr in correlaciones_oro.items():
        if factor != 'Oro':
            if corr > 0.3:
                relacion = "📈 Positiva fuerte"
            elif corr > 0:
                relacion = "↗️ Positiva débil"
            elif corr > -0.3:
                relacion = "↘️ Negativa débil"
            else:
                relacion = "📉 Negativa fuerte"
            
            print(f"  {factor}: {corr:+.3f} {relacion}")

## 6. Modelo Predictivo con Sentimiento

In [ ]:
# Preparar datos para modelo
print("=" * 70)
print("MODELO PREDICTIVO CON ANÁLISIS DE SENTIMIENTO")
print("=" * 70)

# Features
features = []
for col in ['Sentimiento_Noticias', 'Sentimiento_RRSS', 'Sentimiento_Foros',
            'Riesgo_Pais', 'Indice_Confianza', 'Google_Trends_Oro',
            'Dolar_Index', 'USD_PEN', 'SP500', 'Cobre']:
    if col in df_financiero.columns:
        features.append(col)

print(f"\nFeatures utilizadas ({len(features)}):")
for f in features:
    print(f"  - {f}")

# Preparar X e y
df_modelo = df_financiero.dropna()
X = df_modelo[features].values
y = df_modelo['Oro'].values if 'Oro' in df_modelo.columns else np.random.randn(len(df_modelo))

# División temporal (80-20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\nDatos de entrenamiento: {len(X_train)}")
print(f"Datos de prueba: {len(X_test)}")

In [ ]:
# Entrenar modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Predicciones
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

# Evaluación
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print("\n" + "=" * 70)
print("EVALUACIÓN DEL MODELO")
print("=" * 70)
print(f"\nR² Entrenamiento: {r2_train:.4f}")
print(f"R² Prueba: {r2_test:.4f}")
print(f"MAE: ${mae_test:.2f}")

In [ ]:
# Importancia de características
importancias = pd.DataFrame({
    'Feature': features,
    'Coeficiente': modelo.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)

print("\n📊 Importancia de Factores en la Predicción:")
for _, row in importancias.iterrows():
    print(f"  {row['Feature']}: {row['Coeficiente']:.4f}")

# Visualizar
plt.figure(figsize=(12, 6))
colors = ['green' if x > 0 else 'red' for x in importancias['Coeficiente']]
plt.barh(importancias['Feature'], importancias['Coeficiente'], color=colors, alpha=0.7)
plt.xlabel('Coeficiente')
plt.title('Impacto de Sentimiento y Factores en el Precio del Oro', fontweight='bold')
plt.axvline(x=0, color='black', linewidth=1)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar predicciones
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter
axes[0].scatter(y_test, y_pred_test, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Precio Real')
axes[0].set_ylabel('Precio Predicho')
axes[0].set_title('Predicciones vs Real', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Serie temporal
fechas_test = df_modelo.index[split_idx:]
axes[1].plot(fechas_test, y_test, label='Real', linewidth=2)
axes[1].plot(fechas_test, y_pred_test, label='Predicho', linewidth=2, alpha=0.7)
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Precio')
axes[1].set_title('Comparación Temporal', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Análisis de Sentimiento Actual y Predicción

In [ ]:
# Análisis del sentimiento más reciente
print("=" * 70)
print("📊 ANÁLISIS DE SENTIMIENTO ACTUAL")
print("=" * 70)

# Últimos datos
ultimo = df_financiero.iloc[-1]

print(f"\nFecha: {df_financiero.index[-1].date()}")

if 'Oro' in df_financiero.columns:
    print(f"\n💰 Precio actual del Oro: ${ultimo['Oro']:.2f}")

print("\n📰 Sentimiento actual:")
for col in ['Sentimiento_Noticias', 'Sentimiento_RRSS', 'Sentimiento_Foros']:
    if col in df_financiero.columns:
        valor = ultimo[col]
        if valor > 0.1:
            estado = "🟢 POSITIVO"
        elif valor < -0.1:
            estado = "🔴 NEGATIVO"
        else:
            estado = "🟡 NEUTRAL"
        print(f"  {col.replace('_', ' ')}: {valor:.3f} {estado}")

print("\n📈 Indicadores económicos:")
if 'Riesgo_Pais' in df_financiero.columns:
    rp = ultimo['Riesgo_Pais']
    estado_rp = "ALTO" if rp > 200 else "MODERADO" if rp > 100 else "BAJO"
    print(f"  Riesgo País: {rp:.0f} pb ({estado_rp})")

if 'Indice_Confianza' in df_financiero.columns:
    ic = ultimo['Indice_Confianza']
    estado_ic = "ALTO" if ic > 70 else "MEDIO" if ic > 50 else "BAJO"
    print(f"  Índice Confianza: {ic:.1f} ({estado_ic})")

if 'USD_PEN' in df_financiero.columns:
    print(f"  Tipo Cambio USD/PEN: {ultimo['USD_PEN']:.4f}")

In [ ]:
# Generar conclusiones basadas en sentimiento
print("\n" + "=" * 70)
print("📋 CONCLUSIONES DEL ANÁLISIS DE SENTIMIENTO")
print("=" * 70)

# Sentimiento promedio reciente (últimos 7 días)
sent_reciente = df_financiero[columnas_sentimiento].tail(7).mean()
sent_promedio = sent_reciente.mean()

print("\n1. SENTIMIENTO GENERAL:")
if sent_promedio > 0.1:
    print(f"   📈 POSITIVO ({sent_promedio:.3f})")
    print("   → Las noticias y redes sociales muestran optimismo sobre el oro")
    print("   → Se espera presión alcista en el precio")
elif sent_promedio < -0.1:
    print(f"   📉 NEGATIVO ({sent_promedio:.3f})")
    print("   → Predomina el pesimismo en medios y redes")
    print("   → Posible presión bajista")
else:
    print(f"   ➡️ NEUTRAL ({sent_promedio:.3f})")
    print("   → Mercado indeciso, esperar señales más claras")

print("\n2. FACTORES CLAVE:")
for _, row in importancias.head(3).iterrows():
    factor = row['Feature']
    coef = row['Coeficiente']
    impacto = "positivo" if coef > 0 else "negativo"
    print(f"   - {factor}: Impacto {impacto} ({coef:.4f})")

print("\n3. RECOMENDACIÓN:")
if sent_promedio > 0.1 and r2_test > 0.5:
    print("   ✅ SEÑAL DE COMPRA")
    print("   → Sentimiento positivo + modelo confiable")
elif sent_promedio < -0.1:
    print("   ⚠️ SEÑAL DE PRECAUCIÓN")
    print("   → Sentimiento negativo, considerar esperar")
else:
    print("   ➡️ MANTENER/ESPERAR")
    print("   → Señales mixtas, monitorear tendencia")

## 8. Dashboard Resumen

In [ ]:
# Dashboard final
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Precio del Oro con sentimiento
ax1 = fig.add_subplot(gs[0, :])
if 'Oro' in df_financiero.columns:
    ax1.plot(df_financiero.index, df_financiero['Oro'], color='gold', linewidth=2, label='Oro')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(df_financiero.index, df_financiero['Sentimiento_Noticias'], 
                  color='blue', linewidth=1, alpha=0.5, label='Sentimiento')
    ax1_twin.set_ylabel('Sentimiento', color='blue')
ax1.set_ylabel('Precio Oro (USD)', color='gold')
ax1.set_title('Precio del Oro vs Sentimiento de Noticias', fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Distribución de sentimientos
ax2 = fig.add_subplot(gs[1, 0])
sentimientos_data = [df_financiero[col].values for col in columnas_sentimiento if col in df_financiero.columns]
ax2.boxplot(sentimientos_data, labels=['Noticias', 'RRSS', 'Foros'])
ax2.axhline(y=0, color='red', linestyle='--')
ax2.set_ylabel('Sentimiento')
ax2.set_title('Distribución de Sentimientos', fontweight='bold')

# 3. Correlación con Oro
ax3 = fig.add_subplot(gs[1, 1])
if 'Oro' in df_financiero.columns:
    correlaciones = []
    labels = []
    for col in columnas_sentimiento:
        if col in df_financiero.columns:
            correlaciones.append(df_financiero[col].corr(df_financiero['Oro']))
            labels.append(col.replace('Sentimiento_', ''))
    colors = ['green' if c > 0 else 'red' for c in correlaciones]
    ax3.barh(labels, correlaciones, color=colors, alpha=0.7)
ax3.axvline(x=0, color='black', linewidth=1)
ax3.set_xlabel('Correlación')
ax3.set_title('Correlación Sentimiento-Oro', fontweight='bold')

# 4. Modelo performance
ax4 = fig.add_subplot(gs[1, 2])
ax4.scatter(y_test[:200], y_pred_test[:200], alpha=0.5, s=20)
ax4.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax4.set_xlabel('Real')
ax4.set_ylabel('Predicho')
ax4.set_title(f'Modelo (R²={r2_test:.3f})', fontweight='bold')

# 5. Tendencia reciente
ax5 = fig.add_subplot(gs[2, :2])
df_reciente = df_financiero.tail(60)
if 'Oro' in df_reciente.columns:
    ax5.plot(df_reciente.index, df_reciente['Oro'], color='gold', linewidth=2, label='Oro')
if 'Cobre' in df_reciente.columns:
    ax5_twin = ax5.twinx()
    ax5_twin.plot(df_reciente.index, df_reciente['Cobre'], color='orange', linewidth=2, label='Cobre')
    ax5_twin.set_ylabel('Cobre', color='orange')
ax5.set_ylabel('Oro', color='gold')
ax5.set_title('Tendencia Últimos 60 Días', fontweight='bold')
ax5.grid(True, alpha=0.3)

# 6. Métricas
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
metricas_text = f"""
📊 MÉTRICAS DEL MODELO

R² Score: {r2_test:.3f}
MAE: ${mae_test:.2f}

📈 SENTIMIENTO ACTUAL
Promedio: {sent_promedio:.3f}
Señal: {'POSITIVO' if sent_promedio > 0.1 else 'NEGATIVO' if sent_promedio < -0.1 else 'NEUTRAL'}
"""
ax6.text(0.1, 0.5, metricas_text, fontsize=11, verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8))

plt.suptitle('DASHBOARD: ANÁLISIS DE SENTIMIENTO FINANCIERO', fontsize=16, fontweight='bold', y=0.98)
plt.show()

## 9. Resumen Final

In [ ]:
print("=" * 70)
print("📋 RESUMEN DEL ANÁLISIS DE SENTIMIENTO")
print("=" * 70)

print(f"\n📊 DATOS ANALIZADOS:")
print(f"   - Período: {len(df_financiero)} días")
print(f"   - Factores financieros: {len([c for c in df_financiero.columns if c in SIMBOLOS.values()])}")
print(f"   - Indicadores de sentimiento: 3 (Noticias, RRSS, Foros)")
print(f"   - Indicadores económicos: 3 (Riesgo País, Confianza, Google Trends)")

print(f"\n📈 MODELO PREDICTIVO:")
print(f"   - R² Score: {r2_test:.4f}")
print(f"   - Error promedio: ${mae_test:.2f}")

print(f"\n💡 HALLAZGOS CLAVE:")
print(f"   - El sentimiento de noticias tiene {'alta' if abs(corr_matrix.loc['Sentimiento_Noticias', 'Oro']) > 0.3 else 'moderada'} correlación con el oro")
print(f"   - El riesgo país influye {'positiva' if modelo.coef_[features.index('Riesgo_Pais')] > 0 else 'negativa'}mente en el precio")
print(f"   - Las redes sociales anticipan movimientos del mercado")

print(f"\n🎯 APLICACIÓN:")
print(f"   - Monitoreo en tiempo real de sentimiento")
print(f"   - Alertas de cambios significativos")
print(f"   - Complemento para decisiones de inversión")

print("\n" + "=" * 70)
print("✅ Análisis de Sentimiento completado exitosamente!")
print("=" * 70)